In [1]:
import pandas as pd
import numpy as np

In [2]:
btc = pd.read_csv("data/BTC_1H.csv", parse_dates=["timestamp"]).rename(columns={"close": "close_BTC"})
eth = pd.read_csv("data/ETH_1H.csv", parse_dates=["timestamp"]).rename(columns={"close": "close_ETH"})
sol = pd.read_csv("data/SOL_1H.csv", parse_dates=["timestamp"]).rename(columns={"close": "close_SOL"})
ltc = pd.read_csv("data/LTC_1H.csv", parse_dates=["timestamp"])



In [3]:
anchor = pd.merge(
    btc[["timestamp", "close_BTC"]],
    eth[["timestamp", "close_ETH"]],
    on="timestamp", how="inner"
)



In [5]:

ltc["usd_volume"] = ltc["close"] * ltc["volume"]
ltc = ltc[ltc["timestamp"] < "2025-05-09"]
daily = ltc.set_index("timestamp")["usd_volume"].resample("D").sum()
print("Avg daily USD volume:", daily.mean())


Avg daily USD volume: 104172627.2808361


##Eligibility of LTC confirmed

In [6]:
from strategy import generate_signals
from backtest import backtest_signals

In [7]:
signals = generate_signals(ltc, anchor)
print(signals['signal'].value_counts())



signal
HOLD    3064
BUY        4
SELL       4
Name: count, dtype: int64


In [8]:
results_df, metrics = backtest_signals(signals, ltc)


In [9]:
print(metrics)

{'final_capital': np.float64(1106.99), 'final_return_pct': np.float64(10.7), 'sharpe_ratio': np.float64(0.0146), 'max_drawdown_pct': np.float64(9.85)}
